In [1]:
import numpy as np
import pandas as pd

In [2]:
from lightautoml_gpu.reader.gpu.cudf_reader import CudfReader
from lightautoml_gpu.reader.base import PandasToPandasReader

from lightautoml_gpu.transformers.base import SequentialTransformer

from lightautoml_gpu.pipelines.utils import get_columns_by_role

from lightautoml_gpu.transformers.gpu import numeric_gpu, categorical_gpu, datetime_gpu
from lightautoml_gpu.transformers import numeric, categorical, datetime

from lightautoml_gpu.tasks import Task
from lightautoml_gpu.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML

import pandas as pd
import time
from sklearn.model_selection import train_test_split

from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import cudf

In [3]:
features = pd.read_csv('../../data/small_new/multilabel/train Data.csv')
labels = pd.read_csv('../../data/small_new/multilabel/train labels.csv')
labels.drop(columns='id', inplace=True)
data = pd.concat([features, labels], axis=1)
data = data[['n_0000','n_0001','n_0002','n_0003',
             'n_0004','n_0005','n_0006','n_0007',
             'service_a', 'service_b', 'service_c', 
             'service_d', 'service_e', 'service_f',
             'service_g', 'service_h', 'service_i',
             'service_j', 'service_k', 'service_l',
             'service_m', 'service_n']]

tr_data, te_data = train_test_split(
    data, 
    test_size=0.2,  
    random_state=42
)

roles = {
    "target": {'service_a', 'service_b', 'service_c',
               'service_d', 'service_e', 'service_f',
               'service_g', 'service_h', 'service_i',
               'service_j', 'service_k', 'service_l',
               'service_m', 'service_n'},
    "drop" : ['id']
}

/tmp/ipykernel_9002/705777730.py:1: DtypeWarning: Columns (329,331,333,336,338,344,345,346,348,354,355,356,357,358,361,362,364,367,372,377,380,383,385,387,390,392,399,400,406,408,409,413,416,418,419,431,433,437,438,442,448,449,450,453,457,464,473,478,479,481,483,485,486,489,492,495,496,497,498,499,500,503,507,508,510,511,514,515,517,519,520,521,522,523,524,526,527,530,534,537,538,539,541,544,547,548,549,550,551,557,558,560,564,569,572,573,576,577,579,583,596,597,598,600,601,602,604,605,606,608,609,613,619,620,625,627,628,629,631,632,633,636,641,642,643,645,646,647,648,651,655,661,662,665,668,675,676,679,682,685,687,689,690,691,694,698,701,702,703,706,711,712,713,719,720,721,733,735,737,738,742,746,747,748,749,752,754,755,760,764,768,770,781,782,789,797,807,812,814,817,818,822,823,824,825,832,840,843,844,845,850,853,857,858,861,867,868,873,874,876,877,879,880,881,883,886,890,893,897,899,900,901,902,904,905,908,909,910,912,913,914,915,916,922,923,931,933,935,937,939,942,943,946,951,955,9

In [4]:
adv_roles = True

## Imports (for potential use)

In [5]:
# Imports from our package
from lightautoml_gpu.automl.base import AutoML

from lightautoml_gpu.automl.presets.gpu.tabular_gpu_presets import TabularAutoMLGPU, TabularUtilizedAutoMLGPU
from lightautoml_gpu.tasks import Task

from lightautoml_gpu.pipelines.features.gpu.lgb_pipeline_gpu import LGBSimpleFeaturesGPU, LGBAdvancedPipelineGPU
from lightautoml_gpu.pipelines.features.gpu.linear_pipeline_gpu import LinearFeaturesGPU

from lightautoml_gpu.pipelines.features.lgb_pipeline import LGBSimpleFeatures, LGBAdvancedPipeline
from lightautoml_gpu.pipelines.features.linear_pipeline import LinearFeatures


from lightautoml_gpu.ml_algo.gpu.boost_cb_gpu import BoostCBGPU
from lightautoml_gpu.ml_algo.gpu.boost_xgb_gpu import BoostXGB
from lightautoml_gpu.ml_algo.gpu.linear_gpu import LinearLBFGSGPU

from lightautoml_gpu.ml_algo.boost_cb import BoostCB
from lightautoml_gpu.ml_algo.linear_sklearn import LinearLBFGS


from lightautoml_gpu.pipelines.ml.base import MLPipeline
from lightautoml_gpu.pipelines.selection.importance_based import ModelBasedImportanceEstimator, ImportanceCutoffSelector

## TabularAutoML

In [6]:
task = Task('multilabel')

multilabel isn`t supported in lgb


In [ ]:
automl = TabularAutoML(
    task = task, 
    timeout = 3600,
    cpu_limit = 4,
    reader_params = {'n_jobs': 4, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': [['linear_l2', 'cb']]}
)

In [ ]:
oof_pred = automl.fit_predict(data, roles = roles, verbose = 2)

In [7]:
task = Task('multilabel', device='gpu')

multilabel isn`t supported in lgb


In [8]:
automl_gpu = TabularAutoMLGPU(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': [['linear_l2', 'xgb']]}
)

In [9]:
oof_pred_gpu = automl_gpu.fit_predict(data, roles = roles, verbose = 2)

[13:08:41] Stdout logging level is INFO2.
[13:08:41] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[13:08:41] Task: multilabel

[13:08:41] Start automl preset with listed constraints:
[13:08:41] - time: 3600.00 seconds
[13:08:41] - CPU: 1 cores
[13:08:41] - memory: 16 GB

[13:08:41] Train data shape: (14644, 22)
[13:08:42] Feats was rejected during automatic roles guess: []
[13:08:42] Layer 1 train process start. Time left 3599.36 secs
[13:08:42] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[13:08:42] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[13:08:48] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[13:08:52] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[13:08:59] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -7.378517805325207
[13:08:59] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[13:08:59] Time left 3582.19 s

In [10]:
gpu_inf = automl_gpu.predict(data)

In [11]:
automl_gpu.to_cpu()

multilabel isn`t supported in lgb
[13:10:53] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[13:10:53] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[13:10:53] CatBoost uses as obj. MultiCrossEntropy.
altering preselection ml algo
multilabel isn`t supported in lgb
[13:10:54] CatBoost uses as obj. MultiCrossEntropy.
altering preselection empty algo
multilabel isn`t supported in lgb
[13:10:54] CatBoost uses as obj. MultiCrossEntropy.


In [12]:
cpu_inf = automl_gpu.predict(data)

In [13]:
from joblib import dump, load
import time
pickle_file = './gpu.joblib'

In [14]:
start = time.time()
with open(pickle_file, 'wb') as f:
    dump(automl_gpu, f)
raw_dump_duration = time.time() - start
print("Raw dump duration: %0.3fs" % raw_dump_duration)

Raw dump duration: 0.431s


In [15]:

start = time.time()
with open(pickle_file, 'rb') as f:
    model_inf = load(f)
raw_load_duration = time.time() - start
print("Raw load duration: %0.3fs" % raw_load_duration)


Raw load duration: 0.323s


In [16]:
model_inf.levels[0][1]

In [17]:
model_inf.levels[0][1].pre_selection.__dict__

{'features_pipeline': <lightautoml_gpu.pipelines.features.lgb_pipeline.LGBSimpleFeatures at 0x7f409d28dcd0>,
 '_fit_on_holdout': True,
 'ml_algo': <lightautoml_gpu.ml_algo.boost_xgb.BoostXGB at 0x7f409dc899a0>,
 '_empty_algo': None,
 'tuner': <lightautoml_gpu.ml_algo.tuning.base.DefaultTuner at 0x7f409dc890d0>,
 'imp_estimator': <lightautoml_gpu.pipelines.selection.importance_based.ModelBasedImportanceEstimator at 0x7f409dc890a0>,
 '_selected_features': ['n_0007',
  'n_0002',
  'n_0001',
  'n_0006',
  'n_0005',
  'n_0000',
  'n_0004',
  'n_0003'],
 '_in_features': ['n_0002',
  'n_0000',
  'n_0001',
  'n_0003',
  'n_0005',
  'n_0007',
  'n_0004',
  'n_0006'],
 'mapped_importances': n_0007    3.995434
 n_0002    3.209496
 n_0001    1.593635
 n_0006    1.534363
 n_0005    1.505244
 n_0000    1.492427
 n_0004    1.345162
 n_0003    1.319946
 dtype: float64,
 'cutoff': 0}

In [18]:
disk_pred = model_inf.predict(data)

In [19]:
disk_pred.data.T

array([[ 0.04244947, -0.30057025,  0.04992199, ..., -1.3732182 ,
         0.20542504, -0.30290425],
       [-3.948897  , -3.861695  , -3.9606762 , ..., -3.6781282 ,
        -4.013991  , -3.7936115 ],
       [ 2.5633793 ,  2.2378361 ,  2.5335147 , ...,  1.8409401 ,
         2.6185966 ,  2.4816575 ],
       ...,
       [ 0.6087324 ,  0.6681073 ,  0.5795779 , ...,  1.4014516 ,
         0.50451803,  0.88195866],
       [ 1.2875954 ,  1.3330125 ,  1.2504246 , ...,  1.8457365 ,
         1.1895608 ,  1.5126821 ],
       [ 0.95173085,  0.9725757 ,  0.9079424 , ...,  1.5832412 ,
         0.8320736 ,  1.1360058 ]], dtype=float32)

In [20]:
disk_pred.data.T

array([[ 0.04244947, -0.30057025,  0.04992199, ..., -1.3732182 ,
         0.20542504, -0.30290425],
       [-3.948897  , -3.861695  , -3.9606762 , ..., -3.6781282 ,
        -4.013991  , -3.7936115 ],
       [ 2.5633793 ,  2.2378361 ,  2.5335147 , ...,  1.8409401 ,
         2.6185966 ,  2.4816575 ],
       ...,
       [ 0.6087324 ,  0.6681073 ,  0.5795779 , ...,  1.4014516 ,
         0.50451803,  0.88195866],
       [ 1.2875954 ,  1.3330125 ,  1.2504246 , ...,  1.8457365 ,
         1.1895608 ,  1.5126821 ],
       [ 0.95173085,  0.9725757 ,  0.9079424 , ...,  1.5832412 ,
         0.8320736 ,  1.1360058 ]], dtype=float32)

In [21]:
print(cpu_inf.data.T)
print()
print(gpu_inf.data.T)
print()
print(oof_pred_gpu.data.T)
#print()
#print(oof_pred.data.T)
#print()
#print(disk_pred.data.T)

[[ 0.04244947 -0.30057025  0.04992199 ... -1.3732182   0.20542504
  -0.30290425]
 [-3.948897   -3.861695   -3.9606762  ... -3.6781282  -4.013991
  -3.7936115 ]
 [ 2.5633793   2.2378361   2.5335147  ...  1.8409401   2.6185966
   2.4816575 ]
 ...
 [ 0.6087324   0.6681073   0.5795779  ...  1.4014516   0.50451803
   0.88195866]
 [ 1.2875954   1.3330125   1.2504246  ...  1.8457365   1.1895608
   1.5126821 ]
 [ 0.95173085  0.9725757   0.9079424  ...  1.5832412   0.8320736
   1.1360058 ]]

[[2.60331016e-02 2.68223695e-02 2.46186070e-02 ... 5.43985376e-03
  3.22784334e-02 1.89001188e-02]
 [4.08446213e-04 5.33593935e-04 4.10844630e-04 ... 6.44752465e-04
  3.72553157e-04 4.79673443e-04]
 [3.76659691e-01 3.26501429e-01 3.74851704e-01 ... 1.95570916e-01
  3.91054511e-01 3.47043067e-01]
 ...
 [4.95836213e-02 6.57103807e-02 5.31986393e-02 ... 1.40887871e-01
  4.38000150e-02 6.27396554e-02]
 [1.08087435e-01 1.31174549e-01 9.91543308e-02 ... 2.12086126e-01
  9.15241092e-02 1.25935078e-01]
 [8.23267698

In [22]:
cluster = LocalCUDACluster(rmm_managed_memory=True, CUDA_VISIBLE_DEVICES="0",
                               protocol="ucx", enable_nvlink=True,
                               memory_limit="8GB")
print("dashboard:", cluster.dashboard_link)
client = Client(cluster)
client.run(cudf.set_allocator, "managed")

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/comm/ucx.py:61: UserWarning: A CUDA context for device 0 already exists on process ID 9002. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
  warnings.warn(
2023-01-01 13:11:35,803 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-tnj24f20', purging
2023-01-01 13:11:35,804 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-01-01 13:11:35,804 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


dashboard: http://127.0.0.1:8787/status


{'ucx://127.0.0.1:41409': None}

In [ ]:
task = Task('multilabel', device='mgpu')

automl_mgpu = TabularAutoMLGPU(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42, 'npartitions': 2},
    general_params = {'use_algos': [['xgb', 'linear_l2']]},
    client = client
)

oof_pred_mgpu = automl_mgpu.fit_predict(data, roles = roles, verbose = 2)

multilabel isn`t supported in lgb
[13:11:36] CatBoost uses as obj. MultiCrossEntropy.
[13:11:36] Stdout logging level is INFO2.
[13:11:36] Task: multilabel

[13:11:36] Start automl preset with listed constraints:
[13:11:36] - time: 3600.00 seconds
[13:11:36] - CPU: 1 cores
[13:11:36] - memory: 16 GB

[13:11:36] Train data shape: (14644, 22)
[13:11:37] Feats was rejected during automatic roles guess: []
[13:11:37] Layer 1 train process start. Time left 3599.26 secs
[13:11:37] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[13:11:37] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[13:11:41] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[13:11:46] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[13:11:54] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -7.377121890901428
[13:11:54] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[13:11:54] Time left 3582.33 secs

[13:12:15] Selector

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
[13:12:18] task [xgboost.dask-0]:ucx://127.0.0.1:41409 got new rank 0


In [ ]:
automl_mgpu.to_cpu()

In [ ]:
mcpu_inf = automl_mgpu.predict(data)

In [ ]:
print(cpu_inf.data.T)
print()
print(gpu_inf.data.T)
print()
print(oof_pred_gpu.data.T)
print()
#print(oof_pred.data.T)
print()
print(oof_pred_mgpu.data.T)
print()
print(mcpu_inf.data.T)

In [ ]:
from joblib import dump, load
import time

pickle_file = './mgpu.joblib'

start = time.time()
with open(pickle_file, 'wb') as f:
    dump(automl_mgpu, f)
raw_dump_duration = time.time() - start
print("Raw dump duration: %0.3fs" % raw_dump_duration)